In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from scipy.interpolate import griddata

# 1. Load the HYCOM dataset from the specified path
hycom_ds = xr.open_dataset('output.nc')  # Updated path
hycom_surf_el = hycom_ds['surf_el']
hycom_lon = hycom_ds['lon'].values
hycom_lat = hycom_ds['lat'].values

# Adjust HYCOM longitudes from 0-360 to -180 to 180
hycom_lon = np.where(hycom_lon > 180, hycom_lon - 360, hycom_lon)

# 2. Load the ADCIRC dataset from the specified path
adcirc_ds = xr.open_dataset('/scratch/07174/soelem/global_2-20km/fort.63.nc')  # Updated path
adcirc_lon = adcirc_ds['x'].values
adcirc_lat = adcirc_ds['y'].values
adcirc_fill_value = adcirc_ds['zeta'].attrs.get('_FillValue', np.nan)

# 3. Align the time step differences between HYCOM and ADCIRC
# HYCOM starts on 2024-08-15 (index 0) and ADCIRC starts on 2024-08-01.
# We start ADCIRC from index 360 (which is 15 days * 24 hours) and downsample by 3.
adcirc_start_index = 360
adcirc_time_sel = adcirc_ds.isel(time=slice(adcirc_start_index, None, 3))

# 4. Loop over the HYCOM time steps and interpolate ADCIRC data for matching
squared_diff_sum = np.zeros((len(hycom_lat), len(hycom_lon)))  # Placeholder for RMSE computation

for t in range(len(hycom_ds['time'])):
    # Extract HYCOM data for the current time step
    hycom_surf_el_t = hycom_surf_el.isel(time=t).values
    
    # ADCIRC data for the corresponding timestep (adjusted to match HYCOM)
    adcirc_zeta_t = adcirc_time_sel['zeta'].isel(time=t).where(adcirc_time_sel['zeta'] != adcirc_fill_value).values

    # Flatten the ADCIRC zeta values to match the spatial grid (2D)
    adcirc_zeta_t_flat = adcirc_zeta_t[:, 0] if adcirc_zeta_t.ndim == 2 else adcirc_zeta_t

    # Check if the shapes of ADCIRC data and the grid match
    if adcirc_zeta_t_flat.shape[0] != adcirc_lon.shape[0]:
        print(f"Shape mismatch at timestep {t}: ADCIRC zeta shape {adcirc_zeta_t_flat.shape} does not match ADCIRC lon/lat shape {adcirc_lon.shape}")
        continue  # Skip iteration if shape doesn't match

    # Interpolate ADCIRC data to the HYCOM grid using scipy's griddata
    adcirc_interpolated_t = griddata(
        (adcirc_lon, adcirc_lat),  # ADCIRC grid
        adcirc_zeta_t_flat,  # Flatten ADCIRC zeta values
        (hycom_lon[None, :], hycom_lat[:, None]),  # HYCOM grid for interpolation
        method='linear'
    )

    # Handle NaN values in interpolation
    adcirc_interpolated_t = np.ma.masked_invalid(adcirc_interpolated_t)

    # Compute squared difference between HYCOM and interpolated ADCIRC
    squared_diff = (hycom_surf_el_t - adcirc_interpolated_t) ** 2
    squared_diff_sum += np.nan_to_num(squared_diff)

# 5. Compute RMSE by averaging the squared differences and taking the square root
rmse = np.sqrt(squared_diff_sum / len(hycom_ds['time']))

# 6. Plot the RMSE
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Contour plot of RMSE
contour = ax.contourf(hycom_lon, hycom_lat, rmse, 60, cmap='coolwarm', transform=ccrs.PlateCarree())
plt.colorbar(contour, label='RMSE in Water Surface Elevation (m)')

# Add coastlines and land for context
ax.coastlines()
land = cfeature.NaturalEarthFeature('physical', 'land', '110m', edgecolor='face', facecolor='darkgray')
ax.add_feature(land)

plt.title('RMSE between HYCOM and ADCIRC (August 15-30, 2024)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()